In [ ]:
import openpyxl 
import requests
from bs4 import BeautifulSoup
from selenium import webdriver
import re
import pandas as pd
import time
import random
import datetime

In [ ]:
# 날짜 '20210505' 10일 후까지 있음 / 20210407부터 상품상세있음
date = '20211202'

In [ ]:
path = 'https://hsmoa.com/'+'?date='+date
#driver = webdriver.Chrome('../chromedriver')
#ㅋdriver.get(path)
time.sleep(0.5)

In [ ]:
# 정적 파싱
res = requests.get(path)
soup = BeautifulSoup(res.content, 'html.parser')
# 상품 파싱
keywords = list()
#keyword = soup.select('div.before_live')
keyword = soup.select('div.timeline-group')

In [ ]:
dt_index = pd.date_range(start='20211103', end='20211202')
dates = dt_index.strftime("%Y%m%d").tolist()
dates

In [ ]:
product_info_all = pd.DataFrame() # 전체 수집파일 저장

for date in dates:
    path = 'https://hsmoa.com/'+'?date='+date
    #driver = webdriver.Chrome('../chromedriver')
    #driver.get(path)
    time.sleep(0.5)
    # 정적 파싱
    res = requests.get(path)
    soup = BeautifulSoup(res.content, 'html.parser')
    keyword = soup.select('div.timeline-group')

    # 변수선언
    
    name_ls = []
    price_ls = []
    cat_ls = []
    start_time_ls = []
    end_time_ls = []
    timeline_group_ls = []
    company_ls = []
    category_ls = []
    etc_ls = []
    url_ls = []
    id_ls = []
    times_ls = []

    # 1일치 수집에 8분. 
    for data in keyword :
        df1 = data.select('div.timeline-item')
        for j in range(0,len(df1)):
            name = df1[j].select_one('div.font-15').get_text().strip() # 상품명
            #상품명 추출하기
            price = df1[j].select_one('span.strong').get_text() # 금액
            price = re.sub(r'[^0-9]', '', price)    
            #가격 추출하기
            time1 = df1[j].select_one('span.font-12').get_text() # 시간
            time1 = re.sub(r'[^0-9가-힣]', '', time1)
            start_time = time1[0:(time1.find('분')+1)]
            end_time = time1[(time1.find('분')+1):]
            
            company = df1[j].get('class')[1] # 홈쇼핑사
            category = df1[j].get('class')[2] # 카테고리
            #etc = df1[j].get('class')[3] # 기타
            
            timeline_group = df1[j].find('span','font-12').get_text() # 타임라인시간
            timeline_group = re.sub(r'[^0-9가-힣]', '', timeline_group)

            url = df1[j].select_one('a.disblock')['href'] # url
            url = 'https://hsmoa.com' + url

            id = re.sub(r'[^0-9]', '', url)

            # 시간계산
            try:
                end_time_1 = datetime.datetime.strptime(end_time, "%H시%M분")
                start_time_1 = datetime.datetime.strptime(start_time, "%H시%M분")
                times = (end_time_1-start_time_1).seconds/60
            except:
                times = 0 # 계산불가시 0
            
            # 상품명과 가격을 변수에 저장하기
            # keywords.append(product_info)
            # 리스트 변수에 append함수 활용하여 리스트 추가
            name_ls.append(name)
            price_ls.append(price)
            start_time_ls.append(start_time)
            end_time_ls.append(end_time)
            company_ls.append(company)
            category_ls.append(category)
            #etc_ls.append(etc)
            timeline_group_ls.append(timeline_group)
            url_ls.append(url)
            id_ls.append(id)
            times_ls.append(times)

        time.sleep(random.randrange(0,3,1)) # 랜덤시간 슬립

        product_info = pd.DataFrame({'id':id_ls,'name' : name_ls, 'price':price_ls,'start_time':start_time_ls,'end_time':end_time_ls,'company':company_ls,
            'category':category_ls,'timeline_group':timeline_group_ls,'url':url_ls,'times':times_ls}) # 'etc':etc_ls,
    print(date,'수집완료')
    product_info['date'] = date # 날짜추가  
    product_info_all = product_info_all.append(product_info)
product_info_all.to_excel('../result/homemoa_prd_{date}.xlsx'.format(date = date), index=False)

In [ ]:
product_info_all.to_excel('../result/homemoa_prd_{date}.xlsx'.format(date = date), index=False)